# Export commercials

Run this notebook from an esper-tv2 environment.

In [81]:
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
import random
import math
import os

In [42]:
from esperlib.widget import vgrid_widget
from vgrid import VideoBlockFormat
from app.models import Video

video_meta = [Video.objects.filter(id=559).all()[0].for_vgrid()]
vgrid_widget(
    video_meta=video_meta,
    vis_format=VideoBlockFormat(
        imaps=[
            ('test', IntervalSetMapping({
                559: IntervalSet([Interval(Bounds3D(0, 10)), Interval(Bounds3D(20, 30))])
            }))
        ]
    )
)

VGridWidget(vgrid_spec={'settings': {'key_mode': 'Jupyter', 'show_timeline': True, 'spinner_dev_mode': False, …

In [5]:
def get_fps_map(vids):
    vs = Video.objects.filter(id__in=vids)
    return {v.id: v.fps for v in vs}

def frame_second_conversion(c, mode='f2s'):
    fps_map = get_fps_map(set(c.get_grouped_intervals().keys()))
    
    def second_to_frame(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds'].copy()
            curr_bounds['t1'] = int(curr_bounds['t1']*fps)
            curr_bounds['t2'] = int(curr_bounds['t2']*fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    def frame_to_second(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds'].copy()
            curr_bounds['t1'] = int(curr_bounds['t1']/fps)
            curr_bounds['t2'] = int(curr_bounds['t2']/fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    if mode=='f2s':
        fn = frame_to_second
    if mode=='s2f':
        fn = second_to_frame
    output = {}
    for vid, intervals in c.get_grouped_intervals().items():
        output[vid] = intervals.map(fn(fps_map[vid]))
    return IntervalSetMapping(output)

def frame_to_second_collection(c):
    return frame_second_conversion(c, 'f2s')

def second_to_frame_collection(c):
    return frame_second_conversion(c, 's2f')

In [28]:
with open('commercial_gt.pkl', 'rb') as f:
    commercial_gt = pickle.load(f)

In [32]:
video_ids = list(sorted(commercial_gt['all'].keys()))

In [31]:
commercials_ism = IntervalSetMapping({
    vid: IntervalSet([
        Interval(Bounds3D(tup[0], tup[1]))
        for tup in commercial_gt['all'][vid]
    ])
    for vid in commercial_gt['all']
})

In [33]:
vgrid_widget(
    video_meta=[
        Video.objects.get(id=vid).for_vgrid()
        for vid in video_ids
    ],
    vis_format=VideoBlockFormat(imaps = [
        ('all commercials', commercials_ism)
    ]))

VGridWidget(vgrid_spec={'settings': {'key_mode': 'Jupyter', 'show_timeline': True, 'spinner_dev_mode': False, …

# Construct training, validation, test sets

In [46]:
new_keys = list(commercial_gt['new'])
dan_keys = list(commercial_gt['Dan'])
ten_year_keys = list(commercial_gt['10y'])
old_keys = list(commercial_gt['old'])

random.shuffle(new_keys)
random.shuffle(dan_keys)
random.shuffle(ten_year_keys)
random.shuffle(old_keys)

print(len(new_keys))
print(len(dan_keys))
print(len(ten_year_keys))
print(len(old_keys))

29
100
20
61


In [47]:
train_set = new_keys[6:] + dan_keys[20:] + ten_year_keys[4:] + old_keys[12:]
val_set = new_keys[:3] + dan_keys[:10] + ten_year_keys[:2] + old_keys[:6]
test_set = new_keys[3:6] + dan_keys[10:20] + ten_year_keys[2:4] + old_keys[6:12]

In [49]:
print(sorted(train_set))
print(sorted(val_set))
print(sorted(test_set))

[293, 514, 529, 715, 755, 763, 1595, 2648, 3317, 3459, 3769, 3952, 4029, 4143, 4421, 4611, 5281, 6161, 6185, 6532, 7206, 7262, 8220, 8859, 9480, 9499, 9758, 10335, 10621, 11003, 11293, 11555, 11792, 11940, 12655, 13058, 13141, 13247, 13556, 13791, 13827, 13927, 14482, 14638, 15855, 15916, 15964, 16215, 16542, 16599, 16693, 16879, 16964, 17458, 17983, 19882, 19959, 20230, 20380, 20450, 20920, 21572, 23181, 23184, 24193, 24784, 25463, 26060, 26231, 26386, 26824, 26836, 27175, 27188, 27410, 27927, 27963, 28613, 29001, 31378, 31994, 32472, 33004, 33387, 33541, 33977, 34050, 34359, 34642, 36211, 37107, 37113, 37170, 37927, 38275, 38420, 40856, 41235, 41480, 41725, 41836, 42027, 42362, 42756, 44998, 45472, 45573, 45645, 45655, 45698, 45744, 46041, 46058, 46753, 48140, 49225, 49931, 50883, 51145, 51175, 51469, 51482, 51606, 52075, 52749, 53355, 53684, 53932, 54238, 55016, 56051, 56764, 57310, 57384, 57592, 57708, 57798, 57804, 57962, 58389, 59122, 59398, 60186, 60581, 61359, 61930, 62400, 660

# Export Images

In [50]:
train_set = [293, 514, 529, 715, 755, 763, 1595, 2648, 3317, 3459, 3769,
             3952, 4029, 4143, 4421, 4611, 5281, 6161, 6185, 6532, 7206, 
             7262, 8220, 8859, 9480, 9499, 9758, 10335, 10621, 11003, 11293, 
             11555, 11792, 11940, 12655, 13058, 13141, 13247, 13556, 13791, 
             13827, 13927, 14482, 14638, 15855, 15916, 15964, 16215, 16542, 
             16599, 16693, 16879, 16964, 17458, 17983, 19882, 19959, 20230, 
             20380, 20450, 20920, 21572, 23181, 23184, 24193, 24784, 25463, 
             26060, 26231, 26386, 26824, 26836, 27175, 27188, 27410, 27927, 
             27963, 28613, 29001, 31378, 31994, 32472, 33004, 33387, 33541, 
             33977, 34050, 34359, 34642, 36211, 37107, 37113, 37170, 37927, 
             38275, 38420, 40856, 41235, 41480, 41725, 41836, 42027, 42362, 
             42756, 44998, 45472, 45573, 45645, 45655, 45698, 45744, 46041, 
             46058, 46753, 48140, 49225, 49931, 50883, 51145, 51175, 51469, 
             51482, 51606, 52075, 52749, 53355, 53684, 53932, 54238, 55016, 
             56051, 56764, 57310, 57384, 57592, 57708, 57798, 57804, 57962, 
             58389, 59122, 59398, 60186, 60581, 61359, 61930, 62400, 66092, 
             66666, 79265, 80121, 93033, 94663, 112580, 114248, 115653, 
             123531, 124234, 128012, 133584, 134007, 135812, 136446, 148080, 
             158981, 158982, 192899, 205173]
val_set = [559, 1791, 3730, 3754, 10323, 11579, 17386, 20689, 24847, 24992, 
           26175, 33800, 40203, 40267, 43637, 50561, 54377, 57990, 59028, 
           63965, 67300]
test_set = [385, 8697, 9215, 9901, 12837, 13993, 14925, 18700, 23541, 31902,
            32996, 36755, 50164, 52945, 55711, 57748, 59789, 60433, 136732,
            149097, 169420]

In [51]:
import hwang, storehouse

In [70]:
interval = 10
segs_dict = {}
for video_id in video_ids:
    video = Video.objects.get(id=video_id)
    iset = IntervalSet([
        Interval(Bounds3D(i - interval / 2, i + interval / 2))
        for i in range(0, int(video.num_frames / video.fps), interval)
    ])
    segs_dict[video_id] = iset
    
segments = IntervalSetMapping(segs_dict)

In [72]:
segments_frames = second_to_frame_collection(segments)

In [74]:
from PIL import Image
from tqdm import tqdm

In [83]:
for video_id in tqdm(video_ids):
    video = Video.objects.get(id=video_id)
    backend = storehouse.StorageBackend.make_from_config(
        storehouse.StorageConfig.make_gcs_config(os.environ.get('BUCKET')))
    dec = hwang.Decoder(storehouse.RandomReadFile(backend, video.path))
    
    frame_nums = [
        int((intrvl['t1'] + intrvl['t2']) / 2)
        for intrvl in segments_frames.get_grouped_intervals()[video_id].get_intervals()
    ]
    
    frames = dec.retrieve(frame_nums)
    
    os.makedirs('/app/data/commercials/images/{}'.format(video_id), exist_ok=True)
    
    for i, frame in enumerate(frames):
        im = Image.fromarray(frame)
        im.save('/app/data/commercials/images/{}/{:04d}.jpg'.format(video_id, i))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 210/210 [5:57:35<00:00, 121.51s/it]


# Export Labels

In [75]:
segments_all_negative = segments.map(
    lambda intrvl: Interval(intrvl['bounds'], 0)
)

In [78]:
commercial_segments = segments.filter_against(
    commercials_ism, predicate = overlaps(), window = 0
).map(
    lambda intrvl: Interval(intrvl['bounds'], 1)
)

commercial_labels = segments_all_negative.minus(
    commercial_segments, window = 0
).union(commercial_segments)

print(commercial_segments.size())
print(commercial_labels.size())

{514: 120, 4611: 44, 45573: 26, 59398: 106, 41480: 105, 128012: 120, 52749: 190, 11792: 112, 6161: 105, 13827: 123, 38420: 126, 58389: 44, 8220: 124, 11293: 41, 9758: 94, 33541: 107, 158982: 236, 27175: 102, 57384: 119, 6185: 133, 42027: 105, 15916: 108, 4143: 76, 33800: 109, 17458: 117, 27188: 127, 34359: 127, 1595: 106, 26175: 46, 40203: 360, 21572: 118, 114248: 59, 49225: 117, 14925: 82, 10323: 100, 45655: 129, 2648: 256, 7262: 119, 10335: 332, 38275: 131, 529: 103, 60433: 106, 149097: 117, 57962: 119, 53355: 109, 26386: 125, 148080: 105, 43637: 105, 26231: 103, 66666: 105, 24193: 126, 45698: 102, 33387: 118, 135812: 120, 57990: 105, 41836: 121, 123531: 115, 23181: 110, 23184: 127, 31378: 126, 59028: 119, 755: 36, 8859: 45, 51482: 67, 16542: 77, 28613: 131, 46753: 109, 11940: 52, 60581: 119, 136732: 114, 3754: 93, 94663: 111, 53932: 51, 9901: 106, 45744: 119, 3769: 53, 50883: 112, 57798: 101, 26824: 130, 169420: 80, 25463: 130, 24784: 45, 52945: 106, 26836: 55, 16599: 107, 32472: 12

In [82]:
os.makedirs('/app/data/commercials/data', exist_ok=True)
with open('/app/data/commercials/data/train.txt', 'w') as f:
    for video_id in train_set:
        for i, intrvl in enumerate(commercial_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))
with open('/app/data/commercials/data/val.txt', 'w') as f:
    for video_id in val_set:
        for i, intrvl in enumerate(commercial_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))
with open('/app/data/commercials/data/test.txt', 'w') as f:
    for video_id in test_set:
        for i, intrvl in enumerate(commercial_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))